In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rvl-dcip-sample/labels.json
/kaggle/input/rvl-dcip-sample/label2id.json
/kaggle/input/rvl-dcip-sample/id2label.json
/kaggle/input/rvl-dcip-sample/eval_rvl_dcip/state.json
/kaggle/input/rvl-dcip-sample/eval_rvl_dcip/dataset_info.json
/kaggle/input/rvl-dcip-sample/eval_rvl_dcip/data-00000-of-00001.arrow
/kaggle/input/rvl-dcip-sample/train_rvl_dcip/state.json
/kaggle/input/rvl-dcip-sample/train_rvl_dcip/dataset_info.json
/kaggle/input/rvl-dcip-sample/train_rvl_dcip/data-00000-of-00002.arrow
/kaggle/input/rvl-dcip-sample/train_rvl_dcip/data-00001-of-00002.arrow


In [2]:
!pip install -q datasets evaluate

In [3]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [4]:
!pip install -q accelerate -U

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from sklearn.utils import shuffle
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
import evaluate
import json

os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-04-26 10:58:29.893456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 10:58:29.893513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 10:58:29.894994: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
train_dataset = load_from_disk('/kaggle/input/rvl-dcip-sample/train_rvl_dcip')
eval_dataset = load_from_disk('/kaggle/input/rvl-dcip-sample/eval_rvl_dcip')

In [8]:
labels = json.load(open('/kaggle/input/rvl-dcip-sample/labels.json','r'))
id2label = json.load(open('/kaggle/input/rvl-dcip-sample/id2label.json','r'))
label2id = json.load(open('/kaggle/input/rvl-dcip-sample/label2id.json','r'))

In [9]:
train_dataset.set_format(type="torch", device=device)
eval_dataset.set_format(type="torch", device=device)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlmv3-base")


model = AutoModelForSequenceClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                           problem_type="single_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

model.to(device)

Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LayoutLMv3ForSequenceClassification(
  (layoutlmv3): LayoutLMv3Model(
    (embeddings): LayoutLMv3TextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
    )
    (patch_embed): LayoutLMv3PatchEmbeddings(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (encoder): LayoutLMv3Enco

In [11]:
f1_metric = evaluate.load("f1")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1_metric.compute(predictions=predictions, references=labels, average='micro')

In [12]:
batch_size = 8
metric_name = "f1"

args = TrainingArguments(
    "fine-tuned-rvl-cdip",
    logging_dir='./logs',
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:1052: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1
1,No log,2.037663,0.447917
2,No log,1.307528,0.664062
3,No log,0.984980,0.723958
4,No log,0.877549,0.763021
5,No log,0.782431,0.786458
6,1.298700,0.751568,0.802083
7,1.298700,0.768752,0.786458
8,1.298700,0.746214,0.812500
9,1.298700,0.773052,0.817708
10,1.298700,0.775481,0.812500


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:1052: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:1052: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:1052:

TrainOutput(global_step=960, training_loss=0.7810001293818156, metrics={'train_runtime': 3775.8516, 'train_samples_per_second': 4.068, 'train_steps_per_second': 0.254, 'total_flos': 4077172272660480.0, 'train_loss': 0.7810001293818156, 'epoch': 10.0})

In [14]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:1052: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.7730522155761719,
 'eval_f1': 0.8177083333333333,
 'eval_runtime': 35.3763,
 'eval_samples_per_second': 10.855,
 'eval_steps_per_second': 0.678,
 'epoch': 10.0}

In [15]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/davidhajdu/fine-tuned-rvl-cdip/commit/7bf2d436e2745819d47c440c65b9a70495b62901', commit_message='End of training', commit_description='', oid='7bf2d436e2745819d47c440c65b9a70495b62901', pr_url=None, pr_revision=None, pr_num=None)